In [1]:
import pickle

In [2]:
with open('./site_dic.pkl', 'rb') as f:
    site_dict = pickle.load(f)

In [3]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

from time import time

In [4]:
sites = ['site%s' % i for i in range(1, 11)]

In [5]:
df_train = pd.read_csv('train_sessions.csv')

In [8]:
sites = []

for i in range(1, 11):
    s = 'site%d' % i
    df_train[s] = df_train[s].fillna(0).astype(int)
    
    t = 'time%d' % i
    df_train[t] = pd.to_datetime(df_train[t])

    sites.append(s)

In [11]:
df_train = df_train.sort_values(by='time1').reset_index(drop=True)

In [72]:
df = df_train
df['sites'] = df.site1.astype('str') + ' ' + df.site2.astype('str') + ' ' + \
              df.site3.astype('str') + ' ' + df.site4.astype('str') + ' ' + \
              df.site5.astype('str') + ' ' + \
              df.site6.astype('str') + ' ' + df.site7.astype('str') + ' ' + \
              df.site8.astype('str') + ' ' + df.site9.astype('str') + ' ' + \
              df.site10.astype('str')  

In [62]:
columns = df_train[sites].values.flatten()
ones = np.ones_like(columns, dtype=np.uint8)

row_ptrs = np.arange(0, len(ones) + 1, 10)

In [63]:
X = sp.csr_matrix((ones, columns, row_ptrs), )

In [73]:
cv = CountVectorizer(token_pattern='\S+')#, min_df=10, max_df=0.5)
X_ohe = cv.fit_transform(df_train.sites)

In [74]:
X = X_ohe

In [75]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split

In [76]:
y = df_train.target.values

In [77]:
n = len(df_train) // 10

X_train = X[:-n]
y_train = y[:-n]

X_val = X[-n:]
y_val = y[-n:]

In [78]:
for C in [0.01, 0.1, 0.5, 1]:
    t0 = time()

    svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
    #svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.539s, auc=0.877
C=0.1, took 3.197s, auc=0.903
C=0.5, took 6.966s, auc=0.897
C=1, took 7.048s, auc=0.891


In [70]:
for C in [0.01, 0.1, 0.5, 1]:
    t0 = time()

    svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
    #svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.460s, auc=0.877
C=0.1, took 3.189s, auc=0.903
C=0.5, took 6.987s, auc=0.897
C=1, took 7.116s, auc=0.891


In [71]:
for C in [0.01, 0.1, 0.5, 1]:
    t0 = time()

    #svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
    svm = LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
    svm.fit(X_train, y_train)

    y_pred = svm.decision_function(X_val)
    auc = roc_auc_score(y_val, y_pred)

    print('C=%s, took %.3fs, auc=%.3f' % (C, time() - t0, auc))

C=0.01, took 0.594s, auc=0.843
C=0.1, took 0.932s, auc=0.911
C=0.5, took 1.921s, auc=0.912
C=1, took 3.382s, auc=0.911


In [64]:
#C = 0.1
#svm = LinearSVC(penalty='l1', dual=False, C=C, random_state=1)
C = 1
svm =LogisticRegression(penalty='l1', dual=False, C=C, random_state=1)
svm.fit(X, y)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

logreg, tf-idf 1-3 ngrams

- C=1, 0.91868, cv = auc=0.945
- C=5, 0.91512, cv = auc=0.952

In [65]:
df_test = pd.read_csv('test_sessions.csv', dtype={s: 'str' for s in sites})

for s in sites:
    df_test[s].fillna('', inplace=1)

In [66]:
df = df_test
df['sites'] = df.site1 + ' ' + df.site2 + ' ' + df.site3 + ' ' + df.site4 + ' ' + df.site5 + ' ' + \
              df.site6 + ' ' + df.site7 + ' ' + df.site8 + ' ' + df.site9 + ' ' + df.site10  

In [67]:
X_test = cv.transform(df_test.sites)

In [68]:
pred = svm.decision_function(X_test)

In [69]:
df_res = pd.DataFrame()
df_res['session_id'] = df_test.session_id
df_res['target'] = pred

In [70]:
df_res.to_csv('benchmark6.csv', index=False)

In [71]:
!gzip -f benchmark6.csv

- 1 -> LB: 89
- 0.1 -> 0.91416